In [2]:
from tiktoken import encoding_for_model

def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """
    주어진 텍스트의 토큰 수를 계산합니다.
    
    Args:
        text (str): 토큰 수를 계산할 텍스트
        model (str): 사용할 OpenAI 모델 이름 (기본값: gpt-3.5-turbo)
        
    Returns:
        int: 토큰 수
    """
    encoding = encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

def calculate_io_tokens(input_text: str, output_text: str, model: str = "gpt-3.5-turbo"):
    """
    입력과 출력 텍스트의 토큰 수를 계산하고 출력합니다.
    
    Args:
        input_text (str): 입력 텍스트
        output_text (str): 출력 텍스트
        model (str): 사용할 OpenAI 모델 이름 (기본값: gpt-3.5-turbo)
    """
    input_tokens = count_tokens(input_text, model)
    output_tokens = count_tokens(output_text, model)
    total_tokens = input_tokens + output_tokens
    
    print(f"입력 토큰 수: {input_tokens}")
    print(f"출력 토큰 수: {output_tokens}")
    print(f"전체 토큰 수: {total_tokens}")

In [3]:
import pandas as pd

gpt_eval_df = pd.read_excel('evaluation_results/gpt4-judge.xlsx')

instruction = f"""당신은 문제를 채점하는 교사입니다. 

질문, 정답, 그리고 학생 답변이 주어질 것입니다.

채점 기준은 다음과 같습니다:
1. 학생 답변을 정답과 비교하여 사실적 정확성만을 평가합니다.
2. 학생 답변에 상충되는 진술이 포함되지 않았는지 확인하십시오.
3. 학생 답변이 정답보다 더 많은 정보를 포함하더라도, 정답과 비교했을 때 사실적으로 정확하다면 정답으로 평가합니다.

정답 여부:
정답 여부가 True라면, 학생 답변이 모든 채점 기준을 충족했음을 의미합니다.
정답 여부가 False라면, 학생 답변이 모든 채점 기준을 충족하지 않았음을 의미합니다.

답변을 단계별로 설명하여 추론과 결론이 올바른지 확인하십시오.

처음부터 정답을 단순히 말하지 마십시오.
"""
instruction_tokens = count_tokens(instruction)
input_tokens = gpt_eval_df['question'].apply(lambda x: count_tokens(x))
input_tokens = input_tokens + instruction_tokens

# boolean_tokens = gpt_eval_df['judge'].apply(lambda x: count_tokens(x))
output_tokens = gpt_eval_df['explanation'].apply(lambda x: count_tokens(x))
# output_tokens = output_tokens + boolean_tokens

print(sum(input_tokens))
print(sum(output_tokens))
print((sum(input_tokens)/1000*0.00250  + sum(output_tokens)/1000*0.01000) * 5)

5960
2088
0.17890000000000003


### html to markdown

In [2]:
import html2text

def html_to_markdown(html_content):
    # html2text 객체 생성
    markdown_converter = html2text.HTML2Text()

    # 설정: 링크를 단순 텍스트로, 내부 줄바꿈 제거
    markdown_converter.ignore_links = False
    markdown_converter.body_width = 0

    # HTML -> Markdown 변환
    markdown_content = markdown_converter.handle(html_content)

    # 결과 출력
    return markdown_content

In [9]:
import pandas as pd

df = pd.read_excel('documents/QnA_car_sample_30_context_labeling.xlsx')

df = df[df['ground truth page content'].notna()]
df['ground truth page content'] = df['ground truth page content'].apply(html_to_markdown)

df.to_excel('documents/QnA_car_sample_30_context_labeling_markdown.xlsx', index=False)

In [3]:
import json

with open('documents/car_page_upstage_ver.json', 'r') as file:
    data = json.load(file)

data = {k: html_to_markdown(v) for k, v in data.items()}

with open('documents/car_page_upstage_ver_markdown.json', 'w') as file:
    json.dump(data, file, ensure_ascii=False)

In [13]:
import json

results_file = 'result_20250131_q_QnA_car_sample_30_context_labeling_markdown.json'
results_path = f'results/{results_file}'

with open(results_path, 'r') as file:
    results = json.load(file)

for method in results[0]['llm response'].keys():
    print(method)


dense_sts_chunking_500_100_rerank_3
dense_sts_chunking_500_100_no_reranker
sparse_bm25_chunking_500_100_rerank_3
sparse_bm25_chunking_500_100_no_reranker
dense_sts_full_page_rerank_3
dense_sts_full_page_no_reranker
sparse_bm25_full_page_rerank_3
sparse_bm25_full_page_no_reranker


In [19]:
# JSON을 DataFrame으로 변환
data = []
for method in results[0]['llm response'].keys():
    for i in range(len(results)):
        data.append({
        "question": results[i]['question'],
        "ground truth answer": results[i]['ground truth answer'],
        "ground truth page contents": results[i]['ground truth answer context'],
        "method": method,
        "rag answer": results[i]['llm response'][method]['answer'],
        "rag answer page contents": results[i]['llm response'][method]['page contents']
    })

df = pd.DataFrame(data)

In [20]:
df

,question,ground truth answer,ground truth page contents,method,rag answer,rag answer page contents
0,펫케어 특별약관에 가입했는데 저희 반려견 등록번호를 알려드려야 하나요?,네. 보험계약자 또는 피보험자는 이 특별약관의 계약체결 시 가입하고자 하는 반려 동...,보험계약자 또는 피보험자는 이 특별약관의 계약체결 시 가입하고자 하는 반려 \n동...,dense_sts_chunking_500_100_rerank_3,"네, 펫케어 특별약관에 가입할 때 반려동물의 국가동물보호정보시스템 상 동물등록번호를...",[용어풀이\n\n \n\n\n① '폐사'라 함은 보통약관 제21조(보상하는 손해)...
1,자동차보험을 철회할 수 있나요?,일반금융소비자는 보험증권을 받은 날부터 15일과 청약을 한 날부터 30 일 중 먼저...,① 일반금융소비자(*1)는 보험증권을 받은 날부터 15일과 청약을 한 날부터 30 ...,dense_sts_chunking_500_100_rerank_3,"자동차보험을 철회할 수 있는 경우는 특정 조건을 충족해야 합니다. 예를 들어, 장기...",[자동차보험 의무보험 가입대상 차량입니다. \n장기간 자동차를 운행하지 않을 계획...
2,펫케어 특별약관에 가입하려면 제 강아지가 국가동물보호정보시스템에 등록되어야한다는데 ...,국가동물보호시스템'이라 함은 동물보호법 제15조(등록대상동물의 등록 등) 및 동 법...,'국가동물보호시스템'이라 함은 동물보호법 제15조(등록대상동물의 등록 등) 및 동 ...,dense_sts_chunking_500_100_rerank_3,"국가동물보호정보시스템은 농림축산검역본부에서 운영하는 시스템으로, 동물보호법에 따라 ...",[# 3\. 동물등록증\n\n \n\n\n4\. 가족관계증명서 (등록된 반려동물이...
3,전기자동차가 무엇인가요?,이 특별약관에서 전기자동차라 함은 사용연료의 종류가 전기인 자동차로서 「환경 친화적...,이 특별약관에서 전기자동차라 함은 사용연료의 종류가 전기인 자동차로서 「환경 \n...,dense_sts_chunking_500_100_rerank_3,전기자동차란 전기 공급원으로부터 충전 받은 전기에너지를 동력원으로 사용하는 자동차를...,[③ 삭제 \n④ 삭제 \n⑤ 산업통상자원부장관은 제1항에 따라 품질인증을 실시...
4,신규차량을 구입하여 기존 계약의 보험 만기일과 맞추고 싶습니다.,개인이 소유한 2대 이상의 차량을 보험기간의 끝나는 날을 일치시켜 하나의 증권으로 ...,\n\n\n② 위 제1항에도 불구하고 보험기간이 1년 미만이면서 다른 자동차보험...,dense_sts_chunking_500_100_rerank_3,"신규 차량을 구입하신 후 기존 보험 계약의 만기일과 맞추고 싶으시다면, 보험기간이 ...",[# 부하기로 약정한 경우\n\n \n\n\n② 위 제1항에도 불구하고 보험기간이...
...,...,...,...,...,...,...
227,"교통사고로 인해서 장애를 얻으면, 금전적으로 보상받을 수 있나요?",후유장애급별 보상한도 내에서 위자료가 지급됩니다.,# 다. 후유장애\n\n \n\n\n각 보장종목별 보험가입금액 한도 내에서 다음의...,sparse_bm25_full_page_no_reranker,"네, 교통사고로 인해 장애를 얻은 경우, 자동차 보험에서 후유장애 보험금을 받을 수...","[# 자주묻는 질문을 \n확인하세요!\n\n음주운전, 무면허운전, 마약 약물운전 ..."
228,교통사고로 팔이 부러저서 일상생활이 불편합니다. 간병인을 사용하면 보상이 되나요?,보통약관의 「자기신체사고」 또는 「자동차상해 특별약관」을 가입 하고 「간병비지원 특...,이 특별약관은 보통약관의 「자기신체사고」 또는 「자동차상해 특별약관」을 가입 \n...,sparse_bm25_full_page_no_reranker,죄송합니다. 해당 질문에 대한 답변을 찾을 수 없습니다.,"[# 자주묻는 질문을 \n확인하세요!\n\n음주운전, 무면허운전, 마약 약물운전 ..."
229,대물사고 피해자입니다. 자동차수리중 렌터카를 사용할 수 있는 기간은 얼마동안인가요?,최대 30일을 한도로 렌트비용이 인정됩니다.,1\. 렌트비용 인정기간\n\n \n\n\n가. 수리가능한 경우 : 최대 30일을...,sparse_bm25_full_page_no_reranker,대물사고 피해자로서 자동차 수리 중 렌터카를 사용할 수 있는 기간은 수리가능한 경우...,[나. 렌트를 하지 아니하는 경우 : 회사가 지정하는 업체를 통하여 해당 차 \n...
230,병원에 입원 중인 피해자 입니다. 현재 입원중인 병실에 여러명이 함께 있어 여러가지...,피보험자나 피해자의 희망으로 상급병실에 입원하였을 때는 기준병실의 입원료와 상급병실...,다) 피보험자나 피해자의 희망으로 상급병실에 입원하였을 때는 기준병실의 입원료와 상...,sparse_bm25_full_page_no_reranker,죄송합니다. 해당 질문에 대한 답변을 찾을 수 없습니다. 1인실로 옮기는 것에 대한...,[보험자동차를 영리를 목적으로 요금이나 대가를 받고 반복적으로 사용 \n하는 경우...


In [26]:
path = 'evaluation_results/gpt4o-judge_result_20250131_q_QnA_car_sample_30_context_labeling_markdown.xlsx'
df = pd.read_excel(path)
df['correctness'][0]

"(True, '학생 답변은 정답과 비교했을 때 사실적으로 정확합니다. 학생 답변은 반려동물의 국가동물보호정보시스템 상 동물등록번호를 보험회사에 알려주어야 한다는 점을 명확히 하고 있으며, 이는 정답과 일치합니다. 또한, 학생 답변에는 상충되는 진술이 포함되어 있지 않습니다.')"